
# Spotify Sequential Skip Prediction Challenge

### Anna Martignano, Daniele Montesi, ID2223
With  this  project  we  would  like  to  use  Deep  Learning  to  model  a  solution  to  the  challenge  ”  Spotify Sequential  Skip  Prediction  Challenge  -  Predict  if  users  will  skip  or  listen  to  the  music  they’re  streamed” published by Spotify in collaboration with WSDM and CrowdAI.

Citing the CrowdAI Challenge Website:The task is to predict whether individual tracks encountered in a listening session will beskippedby a particular user.  In order to do this, complete information about the first half of a user’slistening session is provided, while the prediction is to be carried out on the second half.  

The output of a prediction is abinary variable for each track in the second half of the session indicating if it was skipped or not, with a 1 indicating that the track skipped, and a 0 indicating that the track was not skipped.The  problem  corresponds  hence  to  a  binary  classification.   However,  the  problem  is  hard  and  must take into account all the history of the user tracks listened, making the model that best fits this problem aRecurrent Neural Networks.


## 0 - Libraries import and useful paths

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import layers, utils, Model, Input
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json, load_model

from pickle import dump
import pickle
from tqdm import tqdm
import string
import numpy as np
import random
import time
import os
import pandas as pd
from glob import glob
from joblib import delayed, Parallel
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GroupKFold, KFold

In [3]:
dirpath = "/Users/danmontesi/Desktop/scalable-machine-learning-deep-learning/Project"
modelpath = dirpath 
datapath = dirpath + "/dataset"
processedpath = datapath 
testpath = datapath 
trainpath = datapath 
featurespath = datapath

print(featurespath)

/Users/danmontesi/Desktop/scalable-machine-learning-deep-learning/Project/dataset


## 1 - Data Exploration

For the sake of order, we will perform the exploration phase on a different notebook:
`SpotifySequentialChallenge - DataExploration`

Here we upload the Training dataset and the Test Dataset.

- The Train and the Test are belonging to the **same day**. This avoid to suffer from the **concept drift phenomena**
- Only a small subset of the training has been employed (the total Spotify Dataset contained 100+GB of data!)
- The day is a Summer day (15 July 2018)

In [22]:
data = pd.DataFrame()

for i in [1]:
  data = pd.concat([data, pd.read_csv(datapath+"/{}.csv".format(i))])

# Setting the flag will be used after we concatenate train-test into one dataset
data['train_flag'] = True

data.head(5)

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,train_flag
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone,True
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone,True
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,...,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone,True
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,False,False,False,True,0,1,...,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone,True
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,False,False,False,True,0,1,...,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone,True


In [23]:
# Test only on a file

test_data_hist  = pd.read_csv(testpath + '/hist_test.csv')
test_data_hist['train_flag']  = False

test_data_input = pd.read_csv(testpath + '/input_test.csv')
test_data_input['train_flag']  = False

In [24]:
# get only a subsample test
chos_sess_ids =  list(set(test_data_hist.session_id))[:100000]
test_data_input = test_data_input[test_data_input.session_id.isin(chos_sess_ids)]
print("First done")
test_data_hist = test_data_hist[test_data_hist.session_id.isin(chos_sess_ids)]
print("Second done")

First done
Second done


In [25]:
data_temp = pd.concat([data,test_data_hist])
data_temp.reset_index(drop=True, inplace=True)

# Feature engineering

We have observed that the  tracks suggested by Spotify are sometimes repeated. We will take that into account encoding that as a feature.

In [26]:
# Check for presence of duplicate tracks into a session
grouped_sess_tracks = data.groupby('session_id')['track_id_clean'].apply(lambda x: x.tolist()).tolist()

In [27]:
session_track_count = data.groupby(['session_id','track_id_clean'], as_index = False).size().reset_index().rename(columns = {0 : 'duplicated_time'})
data = data.merge(session_track_count, on = ['session_id', 'track_id_clean'], how = 'left')
print("On average, the number of duplicate songs in those sessions is --> ")
session_track_count.duplicated_time.mean()

On average, the number of duplicate songs in those sessions is --> 


1.1455476192209029

### We now load the Tracks dataset and joint it with the dataset:

In [29]:
tf = pd.read_csv(featurespath+"/tf_1.csv")
tf = pd.concat([tf, pd.read_csv(featurespath+"/tf_2.csv")])
tf.reset_index(drop=True, inplace=True)

# 2- Dataset Preprocessing
As we can see, the dataset is made of these columns:

- SessionPosition: the position of the track on the session (which can be long up to 20)
- skip_1	skip_2	skip_3	not_skipped: The possible values for skipping. For this purpose, we will drop all of these except for the skip_2, which is the type of skip we are interested on

In [30]:
data["label"] = data["skip_2"]
data.drop(['skip_1', 'skip_2','skip_3', 'not_skipped'], axis=1, inplace=True)

In [31]:

for i in tqdm(data.index):
    hour = int(data.at[i, "hour_of_day"])       
        
    if hour >7 and hour<=13:
        hour = data.at[i, "hour_of_day"] = 0
    elif hour >13 and hour<20:
        hour = data.at[i, "hour_of_day"] = 1
    elif hour >=20:
        hour = data.at[i, "hour_of_day"] = 2
    else:
        hour = data.at[i, "hour_of_day"] = 3

   

100%|██████████| 2990609/2990609 [03:01<00:00, 16521.22it/s]


### Scaling and one-hot-encoding

In [32]:
# Generate dummy columns for categorical variables
# Scaler
print('{0}\t\tNormalising context features...'.format(datetime.datetime.now()))
features = list(set(data.columns) - set(['session_id','track_id_clean','date', 'label', 'session_position', 'train_flag', "context_switch", 'session_length', 'no_pause_before_play','short_pause_before_play','long_pause_before_play','hist_user_behavior_n_seekfwd','hist_user_behavior_is_shuffle','premium','hour_of_day', 'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end', 'session_length', 'context_type']))
data[features] = StandardScaler().fit_transform(data[features])

features_t = list(set(tf.columns) - set(['track_id', 'time_signature', 'mode', 'key']))
tf[features_t] = StandardScaler().fit_transform(tf[features_t])

features.append('label')
features.append('session_position')

print('{0}\t\tOne-hot-encoding categorical variables...'.format(datetime.datetime.now()))
data = pd.get_dummies(data, columns = ['context_type'], prefix = ['context_type'])
data = pd.get_dummies(data, columns = ['hist_user_behavior_reason_start'], prefix = ['hist_user_behavior_reason_start'])
data = pd.get_dummies(data, columns = ['hist_user_behavior_reason_end'], prefix = ['hist_user_behavior_reason_end'])
data = pd.get_dummies(data, columns = ['hour_of_day'], prefix = ['hour_of_day'])
# Features to use

tf = pd.get_dummies(tf, columns = ['mode'], prefix = ['mode'])
tf = pd.get_dummies(tf, columns = ['key'], prefix = ['key'])
tf = pd.get_dummies(tf, columns = ['time_signature'], prefix = ['time_signature'])


2020-01-12 13:21:22.495396		Normalising context features...
2020-01-12 13:22:18.048885		One-hot-encoding categorical variables...


In [33]:
missing_tdi_feat = set(data.columns.values) - set(test_data_input.columns.values) 
for f in list(missing_tdi_feat):
    test_data_input[f] = 0.0

In [34]:
data = pd.concat([data,test_data_input], sort=False)
data.fillna(.0,inplace=True)

# 3- Dataset Creation

The dataset is formed analogously to whast reported in the 5th place solution paper: http://adrem.uantwerpen.be//bibrem/pubs/WSDMCupJeunen2019.pdf

![alt test](pics/model.png)

(https://github.com/olivierjeunen/sequential-skip-prediction/blob/master/src/RNN.py in generate_model() )

where Xh and Xf are respectively 2 matrices for the session history and future. 
### The steps to the dataset creation are:

1. Merge track features and session features
2. Split based on the time, and drop that feature (with other useless features)
3. create the History-Matrix and Future Matrix. Create duplicate files for each new song to predict its label 

## Keep only tracks present in the training set

Then, we will explore how frequent they are :)

In [35]:
set_tracks_present = set(data.track_id_clean.values)
tracks_set = set(tf.track_id)
print("There are", len(set_tracks_present), "tracks in the dataset \nInstead, in the tracks_data there are", len(tracks_set) )
print("Missing tracks on dataset:", len(set(set_tracks_present)-tracks_set)*100/len(set(set_tracks_present)), "%")

There are 370581 tracks in the dataset 
Instead, in the tracks_data there are 3706388
Missing tracks on dataset: 0.0 %


In [36]:
#TODO Correct: must output a number
data["hist_user_behavior_is_shuffle"] = data['hist_user_behavior_is_shuffle'].astype(np.int64)
data["premium"] = data['premium'].astype(np.int64)
data["label"] = data['label'].astype(np.int64)

In [37]:
# Sorting by session & position of track in session
data.sort_values(['session_id','session_position', "date"], inplace = True)

In [38]:
tf["track_id_clean"] = tf["track_id"]
tf.drop(["track_id"], axis=1, inplace=True)

data.drop("date", axis=1, inplace=True)

In [39]:
dataset = data.merge(tf, on = 'track_id_clean', how = 'left')

# The Last Step: Feature engineering:

Feature engineering is the key for winning competitions of Data Science. Now that we have the whole dataset we are going to compute:
1. Popularity of the songs liked/disliked
2. Duration of songs liked/disliked
3. Release date estimate
4. Acoustic vector correlation


Following the work of the 7th place solution, we are going to first, select some tracks:
- Positive tracks (i.e. the ones that are listened by the user)
- Negative tracks (i.e. the ones that are skipped by the user)
- Song considered' feature

Then, we are saving the features as the DELTA between the
- positive_feat - song-feat
- negative-feat - song_feat

Where positive_feat is the feature (i.e. populatiry) **belonging to ALL the songs LIKED by the user, averaged**.

Moreover, we are going to compute the **dot product** between the acoustic vector of the song and the one of the positive/negative tracks, trying to see whether there is a correlation between the 2.


As addiction, we are computing some similarities of those (Cosine Similarity, Manhattan Distance, ...)

We are choosing to compute Cosine Similarity


In [82]:
train_data = dataset[dataset.train_flag==True]
train_data.sort_values(['session_id','session_position'], inplace = True)
train_data.reset_index(drop=True, inplace=True)
train_data.head(35)

/Users/danmontesi/.conda/envs/ScientificWriting/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,session_id,session_position,session_length,track_id_clean,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,...,key_8,key_9,key_10,key_11,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,pos_duration
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,0.0,0.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,1,0,0,0,0,1,0,NaN
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0,0,0,0,0,1,0,NaN
5,0_00006f66-33e5-4de7-a324-2d18e439fc1e,6,20,t_c815228b-3212-4f9e-9d4f-9cb19b248184,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,0,1,NaN
6,0_00006f66-33e5-4de7-a324-2d18e439fc1e,7,20,t_e23c19f5-4c32-4557-aa44-81372c2e3705,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
7,0_00006f66-33e5-4de7-a324-2d18e439fc1e,8,20,t_0be6eced-f56f-48bd-8086-f2e0b760fdee,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
8,0_00006f66-33e5-4de7-a324-2d18e439fc1e,9,20,t_f3ecbd3b-9e8e-4557-b8e0-39cfcd7e65dd,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN
9,0_00006f66-33e5-4de7-a324-2d18e439fc1e,10,20,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,0.0,1.0,0.0,0.0,0.0,-0.077677,...,0,0,0,0,0,0,0,1,0,NaN


In [101]:
toprocess_dataset = train_data[:1000011]

In [63]:
features = ['us_popularity_estimate', 'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness',
       'liveness', 'loudness', 'mechanism','tempo' , 'organism', 'speechiness', 'valence']



def extract_features_session_track(song, positive_tracks, negative_tracks):
    mean_duration_pos = (positive_tracks['duration'] - song['duration']).mean()
    mean_duration_neg = (negative_tracks['duration'] - song['duration']).mean()
    year_pos = (positive_tracks['release_year'] - song['release_year']).mean()
    year_neg = (negative_tracks['release_year'] - song['release_year']).mean()
    pop_pos = (positive_tracks['us_popularity_estimate'] - song['us_popularity_estimate']).mean()
    pop_neg = (negative_tracks['us_popularity_estimate'] - song['us_popularity_estimate']).mean()
 
    latent_vectors = ['acoustic_vector_0', 'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3', 
                      'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7']
    mean_dot_vector_pos = positive_tracks[latent_vectors].dot(song[latent_vectors]).mean()
    mean_dot_vector_neg = negative_tracks[latent_vectors].dot(song[latent_vectors]).mean()

    return {'pos_duration': mean_duration_pos, 
        'neg_duration': mean_duration_neg,
        'pos_year': year_pos,
        'neg_year': year_neg,
        'pop_neg': pop_neg,
        'pop_pos': pop_pos,
        'pos_dot': mean_dot_vector_pos,
        'neg_dot': mean_dot_vector_neg
        }


In [41]:
# Using tf with the track_id as index (for the feature engineering part)
tf_old = tf.copy()
tf.set_index('track_id_clean', inplace=True)

In [102]:
# TRAINING DATA PROCESSING

current_index = 0
num_lines_completed = 0
completed_index = 0



# Initialize the list
dict_feat = {'pos_duration': [], 
        'neg_duration': [],
        'pos_year': [],
        'neg_year': [],
        'pop_neg': [],
        'pop_pos': [],
        'pos_dot': [],
        'neg_dot': []
        }

# Iterating over the whole dataset:

pbar = tqdm(total=len(toprocess_dataset)-1)

while current_index < len(toprocess_dataset)-1:

    partial_length = toprocess_dataset['session_length'].iloc[current_index]-toprocess_dataset['session_position'].iloc[current_index]+1
    half = int(np.floor(partial_length / 2.0))
    last_session_tracks = toprocess_dataset.loc[current_index+(half):current_index+partial_length-1]

    first_session_tracks = toprocess_dataset.loc[current_index:current_index+(half)-1]
    last_session_item = first_session_tracks.iloc[-1]

    skipped = first_session_tracks[first_session_tracks['label'] == 1]['track_id_clean']
    completed = first_session_tracks[first_session_tracks['label'] == 0]['track_id_clean']

    skipped = tf.loc[skipped]
    completed = tf.loc[completed]

    # Put at zero the new engineered features
    for k,v in dict_feat.items():
        dict_feat[k] = dict_feat[k]+[0]*len(first_session_tracks)

    for j, session_track_row in last_session_tracks.iterrows():
        song_row = tf.loc[session_track_row['track_id_clean']]

        session_track_features = extract_features_session_track(song_row, completed, skipped)

        # appending to the whole feature dictionary
        for k,v in session_track_features.items():
            dict_feat[k] = dict_feat[k]+[v]

    current_index += partial_length
    pbar.update(partial_length)

pbar.close()

for k,v in dict_feat.items():
    toprocess_dataset.loc[:, k] = v
    








  0%|          | 0/1000010 [00:00<?, ?it/s]






  0%|          | 40/1000010 [00:00<57:48, 288.34it/s]






  0%|          | 80/1000010 [00:00<1:00:04, 277.44it/s]






  0%|          | 106/1000010 [00:00<1:08:05, 244.74it/s]






  0%|          | 141/1000010 [00:00<1:02:16, 267.61it/s]






  0%|          | 181/1000010 [00:00<1:00:06, 277.23it/s]






  0%|          | 221/1000010 [00:00<1:08:45, 242.36it/s]






  0%|          | 260/1000010 [00:01<1:15:20, 221.16it/s]






  0%|          | 290/1000010 [00:01<1:09:50, 238.57it/s]






  0%|          | 327/1000010 [00:01<1:07:31, 246.77it/s]






  0%|          | 367/1000010 [00:01<1:22:15, 202.55it/s]






  0%|          | 395/1000010 [00:01<1:27:50, 189.67it/s]






  0%|          | 421/1000010 [00:01<1:24:51, 196.33it/s]






  0%|          | 453/1000010 [00:02<1:22:37, 201.61it/s]






  0%|          | 476/1000010 [00:02<1:29:20, 186.47it/s]






  0%|          | 496/1000010 [00:02<1:30:46, 183.52it/s]






  

  0%|          | 3182/1000010 [00:14<1:10:44, 234.86it/s]






  0%|          | 3222/1000010 [00:14<1:11:45, 231.51it/s]






  0%|          | 3254/1000010 [00:15<1:21:12, 204.57it/s]






  0%|          | 3285/1000010 [00:15<1:13:53, 224.82it/s]






  0%|          | 3325/1000010 [00:15<1:07:14, 247.05it/s]






  0%|          | 3359/1000010 [00:15<1:08:18, 243.17it/s]






  0%|          | 3397/1000010 [00:15<1:13:02, 227.40it/s]






  0%|          | 3435/1000010 [00:15<1:09:46, 238.04it/s]






  0%|          | 3468/1000010 [00:16<1:09:58, 237.34it/s]






  0%|          | 3510/1000010 [00:16<1:11:10, 233.35it/s]






  0%|          | 3550/1000010 [00:16<1:17:14, 215.00it/s]






  0%|          | 3590/1000010 [00:16<1:16:40, 216.57it/s]






  0%|          | 3630/1000010 [00:16<1:18:35, 211.29it/s]






  0%|          | 3663/1000010 [00:16<1:18:00, 212.85it/s]






  0%|          | 3698/1000010 [00:17<1:13:37, 225.52it/s]






  0%|          | 3726/1000010 [00:17<1:1

  1%|          | 6137/1000010 [00:30<1:52:36, 147.10it/s]






  1%|          | 6165/1000010 [00:30<1:44:09, 159.04it/s]






  1%|          | 6185/1000010 [00:30<1:47:20, 154.31it/s]






  1%|          | 6205/1000010 [00:30<1:46:06, 156.09it/s]






  1%|          | 6225/1000010 [00:30<1:39:49, 165.92it/s]






  1%|          | 6248/1000010 [00:31<1:40:25, 164.92it/s]






  1%|          | 6268/1000010 [00:31<1:37:07, 170.54it/s]






  1%|          | 6304/1000010 [00:31<1:26:52, 190.64it/s]






  1%|          | 6344/1000010 [00:31<1:26:53, 190.58it/s]






  1%|          | 6373/1000010 [00:31<1:31:29, 181.00it/s]






  1%|          | 6398/1000010 [00:31<1:25:25, 193.85it/s]






  1%|          | 6438/1000010 [00:31<1:17:28, 213.76it/s]






  1%|          | 6478/1000010 [00:32<1:14:35, 221.98it/s]






  1%|          | 6513/1000010 [00:32<1:20:15, 206.30it/s]






  1%|          | 6543/1000010 [00:32<1:15:47, 218.47it/s]






  1%|          | 6583/1000010 [00:32<1:1

  1%|          | 8923/1000010 [00:45<1:15:00, 220.20it/s]






  1%|          | 8954/1000010 [00:45<1:13:57, 223.31it/s]






  1%|          | 8983/1000010 [00:45<1:17:30, 213.10it/s]






  1%|          | 9020/1000010 [00:46<1:21:52, 201.73it/s]






  1%|          | 9053/1000010 [00:46<1:23:16, 198.32it/s]






  1%|          | 9093/1000010 [00:46<1:26:11, 191.63it/s]






  1%|          | 9117/1000010 [00:46<1:23:28, 197.86it/s]






  1%|          | 9146/1000010 [00:46<1:37:09, 169.97it/s]






  1%|          | 9180/1000010 [00:47<1:48:13, 152.59it/s]






  1%|          | 9200/1000010 [00:47<1:45:22, 156.71it/s]






  1%|          | 9220/1000010 [00:47<1:49:07, 151.33it/s]






  1%|          | 9240/1000010 [00:47<1:42:22, 161.30it/s]






  1%|          | 9268/1000010 [00:47<1:46:16, 155.37it/s]






  1%|          | 9287/1000010 [00:47<1:50:26, 149.50it/s]






  1%|          | 9319/1000010 [00:47<1:38:52, 167.01it/s]






  1%|          | 9339/1000010 [00:48<1:3

  1%|          | 11911/1000010 [01:00<1:15:18, 218.65it/s]






  1%|          | 11937/1000010 [01:00<1:20:25, 204.75it/s]






  1%|          | 11962/1000010 [01:00<1:18:45, 209.09it/s]






  1%|          | 11996/1000010 [01:00<1:15:51, 217.10it/s]






  1%|          | 12033/1000010 [01:01<1:14:45, 220.25it/s]






  1%|          | 12065/1000010 [01:01<1:15:06, 219.22it/s]






  1%|          | 12100/1000010 [01:01<1:12:10, 228.15it/s]






  1%|          | 12140/1000010 [01:01<1:07:36, 243.52it/s]






  1%|          | 12180/1000010 [01:01<1:08:40, 239.72it/s]






  1%|          | 12212/1000010 [01:01<1:06:18, 248.30it/s]






  1%|          | 12245/1000010 [01:02<1:07:20, 244.49it/s]






  1%|          | 12285/1000010 [01:02<1:08:54, 238.89it/s]






  1%|          | 12318/1000010 [01:02<1:06:18, 248.26it/s]






  1%|          | 12346/1000010 [01:02<1:04:16, 256.12it/s]






  1%|          | 12381/1000010 [01:02<1:06:32, 247.38it/s]






  1%|          | 12412/10

...

 23%|██▎       | 229055/1000010 [41:55<3:31:40, 60.70it/s]






 23%|██▎       | 229075/1000010 [41:55<3:18:17, 64.80it/s]






 23%|██▎       | 229095/1000010 [41:55<3:17:13, 65.15it/s]






 23%|██▎       | 229111/1000010 [41:55<3:10:54, 67.30it/s]






 23%|██▎       | 229126/1000010 [41:56<3:17:46, 64.96it/s]






 23%|██▎       | 229146/1000010 [41:56<3:19:57, 64.25it/s]






 23%|██▎       | 229160/1000010 [41:56<3:12:35, 66.71it/s]






 23%|██▎       | 229180/1000010 [41:56<3:15:54, 65.58it/s]






 23%|██▎       | 229200/1000010 [41:57<3:14:38, 66.00it/s]






 23%|██▎       | 229220/1000010 [41:57<3:10:37, 67.39it/s]






 23%|██▎       | 229240/1000010 [41:57<3:15:53, 65.58it/s]






 23%|██▎       | 229260/1000010 [41:58<3:10:39, 67.38it/s]






 23%|██▎       | 229280/1000010 [41:58<3:13:40, 66.32it/s]






 23%|██▎       | 229300/1000010 [41:58<3:14:34, 66.01it/s]






 23%|██▎       | 229320/1000010 [41:59<3:11:51, 66.95it/s]






 23%|██▎       | 229331/1

 23%|██▎       | 230661/1000010 [42:20<2:57:57, 72.06it/s]






 23%|██▎       | 230676/1000010 [42:21<3:08:38, 67.97it/s]






 23%|██▎       | 230688/1000010 [42:21<3:01:13, 70.75it/s]






 23%|██▎       | 230700/1000010 [42:21<2:55:34, 73.03it/s]






 23%|██▎       | 230717/1000010 [42:21<2:59:35, 71.39it/s]






 23%|██▎       | 230737/1000010 [42:22<2:55:10, 73.19it/s]






 23%|██▎       | 230757/1000010 [42:22<2:53:55, 73.72it/s]






 23%|██▎       | 230777/1000010 [42:22<3:06:27, 68.76it/s]






 23%|██▎       | 230797/1000010 [42:22<3:19:34, 64.23it/s]






 23%|██▎       | 230807/1000010 [42:23<4:02:47, 52.80it/s]






 23%|██▎       | 230827/1000010 [42:23<4:25:40, 48.25it/s]






 23%|██▎       | 230842/1000010 [42:23<4:07:59, 51.69it/s]






 23%|██▎       | 230852/1000010 [42:24<4:04:03, 52.52it/s]






 23%|██▎       | 230872/1000010 [42:24<3:53:29, 54.90it/s]






 23%|██▎       | 230892/1000010 [42:24<3:35:08, 59.58it/s]






 23%|██▎       | 230910/1

 23%|██▎       | 232267/1000010 [42:48<3:46:10, 56.58it/s]






 23%|██▎       | 232277/1000010 [42:48<3:59:30, 53.43it/s]






 23%|██▎       | 232297/1000010 [42:48<3:43:49, 57.17it/s]






 23%|██▎       | 232314/1000010 [42:48<3:50:39, 55.47it/s]






 23%|██▎       | 232326/1000010 [42:49<3:48:16, 56.05it/s]






 23%|██▎       | 232344/1000010 [42:49<3:40:36, 58.00it/s]






 23%|██▎       | 232361/1000010 [42:49<3:51:27, 55.28it/s]






 23%|██▎       | 232375/1000010 [42:50<3:41:47, 57.68it/s]






 23%|██▎       | 232393/1000010 [42:50<3:37:43, 58.76it/s]






 23%|██▎       | 232404/1000010 [42:50<3:53:31, 54.78it/s]






 23%|██▎       | 232424/1000010 [42:50<3:44:13, 57.05it/s]






 23%|██▎       | 232442/1000010 [42:51<3:46:29, 56.48it/s]






 23%|██▎       | 232462/1000010 [42:51<3:39:36, 58.25it/s]






 23%|██▎       | 232474/1000010 [42:51<3:47:13, 56.30it/s]






 23%|██▎       | 232494/1000010 [42:52<3:45:54, 56.62it/s]






 23%|██▎       | 232512/1

 23%|██▎       | 233875/1000010 [43:16<4:16:32, 49.77it/s]






 23%|██▎       | 233895/1000010 [43:17<4:05:21, 52.04it/s]






 23%|██▎       | 233906/1000010 [43:17<4:16:44, 49.73it/s]






 23%|██▎       | 233923/1000010 [43:17<4:20:30, 49.01it/s]






 23%|██▎       | 233934/1000010 [43:17<4:23:15, 48.50it/s]






 23%|██▎       | 233948/1000010 [43:18<4:20:43, 48.97it/s]






 23%|██▎       | 233968/1000010 [43:18<4:13:45, 50.31it/s]






 23%|██▎       | 233987/1000010 [43:18<4:13:36, 50.34it/s]






 23%|██▎       | 233999/1000010 [43:19<4:24:49, 48.21it/s]






 23%|██▎       | 234011/1000010 [43:19<4:15:09, 50.03it/s]






 23%|██▎       | 234031/1000010 [43:19<4:10:30, 50.96it/s]






 23%|██▎       | 234051/1000010 [43:20<4:07:42, 51.54it/s]






 23%|██▎       | 234065/1000010 [43:20<4:04:08, 52.29it/s]






 23%|██▎       | 234085/1000010 [43:20<4:07:33, 51.56it/s]






 23%|██▎       | 234102/1000010 [43:21<4:00:16, 53.13it/s]






 23%|██▎       | 234122/1

KeyboardInterrupt: 

In [107]:
toprocess_dataset = toprocess_dataset[:235322]
for k,v in dict_feat.items():
    toprocess_dataset.loc[:, k] = v 

In [109]:
toprocess_dataset = toprocess_dataset[:235306]

,session_id,session_position,session_length,track_id_clean,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,...,time_signature_4,time_signature_5,pos_duration,neg_duration,pos_year,neg_year,pop_neg,pop_pos,pos_dot,neg_dot
235286,0_1460d537-0dc6-4b17-8097-9d3255e2c967,8,10,t_20db6cf1-61e3-4a3f-97ad-08da0930d934,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,-0.499338,-0.071802,-0.067979,-0.090638,-0.057894,-0.199746,7.275965,6.301256
235287,0_1460d537-0dc6-4b17-8097-9d3255e2c967,9,10,t_b3dad6bf-7ecc-48e7-aac3-87b246fe3bc4,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,-0.271099,0.156437,0.203936,0.181276,0.073054,-0.068799,6.995149,6.309045
235288,0_1460d537-0dc6-4b17-8097-9d3255e2c967,10,10,t_de0a2e19-2eda-4dba-bf93-44bbb6c33445,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,-0.185155,0.242381,0.022660,0.000000,0.140441,-0.001411,6.936147,6.279409
235289,0_14611ba3-8944-46fc-be24-86775ef7d6f0,1,17,t_300b9c3c-8577-40cf-8528-1c8e9374112b,0.0,0.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235290,0_14611ba3-8944-46fc-be24-86775ef7d6f0,2,17,t_1051bc37-1a73-4301-812b-f83ae0c9bbcd,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235291,0_14611ba3-8944-46fc-be24-86775ef7d6f0,3,17,t_5823c585-efc4-4759-bbf8-ffe171992727,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235292,0_14611ba3-8944-46fc-be24-86775ef7d6f0,4,17,t_1051bc37-1a73-4301-812b-f83ae0c9bbcd,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235293,0_14611ba3-8944-46fc-be24-86775ef7d6f0,5,17,t_300b9c3c-8577-40cf-8528-1c8e9374112b,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235294,0_14611ba3-8944-46fc-be24-86775ef7d6f0,6,17,t_87e4ce4c-6b35-4aed-9c6b-bbe3acc02a8a,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235295,0_14611ba3-8944-46fc-be24-86775ef7d6f0,7,17,t_5e235989-6bf5-4a0a-842f-8289876b0ce1,0.0,1.0,0.0,0.0,0.0,-0.077677,...,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [110]:
dataset_train = toprocess_dataset.copy()

In [111]:
test_data = dataset[dataset.train_flag==False]
test_data.sort_values(['session_id','session_position'], inplace = True)
test_data.reset_index(drop=True, inplace=True)

toprocess_dataset = test_data

/Users/danmontesi/.conda/envs/ScientificWriting/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [127]:
# TEST DATA PROCESSING

current_index = 0
num_lines_completed = 0
completed_index = 0



# Initialize the list
dict_feat = {'pos_duration': [], 
        'neg_duration': [],
        'pos_year': [],
        'neg_year': [],
        'pop_neg': [],
        'pop_pos': [],
        'pos_dot': [],
        'neg_dot': []
        }

# Iterating over the whole dataset:

pbar = tqdm(total=len(toprocess_dataset)-1)

while current_index < len(toprocess_dataset)-1:

    partial_length = toprocess_dataset['session_length'].iloc[current_index]-toprocess_dataset['session_position'].iloc[current_index]+1
    half = int(np.floor(partial_length / 2.0))
    last_session_tracks = toprocess_dataset.loc[current_index+(half):current_index+partial_length-1]

    first_session_tracks = toprocess_dataset.loc[current_index:current_index+(half)-1]
    last_session_item = first_session_tracks.iloc[-1]
    print(len(first_session_tracks))

    skipped = first_session_tracks[first_session_tracks['label'] == 1]['track_id_clean']
    completed = first_session_tracks[first_session_tracks['label'] == 0]['track_id_clean']

    skipped = tf.loc[skipped]
    completed = tf.loc[completed]

    # Put at zero the new engineered features
    for k,v in dict_feat.items():
        dict_feat[k] = dict_feat[k]+[0]*len(first_session_tracks)

    for j, session_track_row in last_session_tracks.iterrows():
        song_row = tf.loc[session_track_row['track_id_clean']]

        session_track_features = extract_features_session_track(song_row, completed, skipped)

        # appending to the whole feature dictionary
        for k,v in session_track_features.items():
            dict_feat[k] = dict_feat[k]+[v]

    current_index += partial_length
    pbar.update(partial_length)

pbar.close()

for k,v in dict_feat.items():
    toprocess_dataset.loc[:, k] = v












  0%|          | 0/374344 [00:00<?, ?it/s]










  0%|          | 25/374344 [00:00<25:46, 242.10it/s]

5
2
5
4
4
4













  0%|          | 50/374344 [00:00<25:51, 241.19it/s]










  0%|          | 80/374344 [00:00<24:47, 251.66it/s]

2
4
5
3
5
5
5
5













  0%|          | 120/374344 [00:00<22:49, 273.22it/s]










  0%|          | 159/374344 [00:00<21:50, 285.43it/s]

...












  1%|          | 3442/374344 [00:13<28:59, 213.27it/s]










  1%|          | 3469/374344 [00:13<27:24, 225.52it/s]

2
3
5
5
4
5













  1%|          | 3493/374344 [00:14<27:21, 225.91it/s]

3
5


KeyboardInterrupt: 

In [123]:
len_feat = len(dict_feat['neg_year'])
toprocess_dataset_test = toprocess_dataset[:len_feat]
for k,v in dict_feat.items():
    toprocess_dataset.loc[:, k] = v 

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
dataset = pd.concat([dataset_train, toprocess_dataset])

## Organize the dataset into samples. 

Let n be the maximal session length. With k = ⌊n/2].

As session lengths can vary, shorter sessions are padded with zeroes to match the size of the longest ones. Historical tracks Xh are padded before the track sequence, and future tracks Xf are padded after the track sequence.

Input of the network is made of 2 np.arrays-matrices:
- **session_histories** old_matrix R^ (f x k x t), where k is the first half of the features. Made of all the features of thefirst half of the tracks
- **session_futures** new_matrix: R^ (f x (n-k) x t), where n is the total length of features. It is made of all the features in the second half of the tracks (without their labels)

To do so, create 4 lists containing:
1. session_histories as np.array
2. session_futures as np.array
3. session_labels as lists
4. sizes as list of int

In [130]:
max_seq_len = 20

def process_sessions(data, sess_features, track_feature_list, label_index):
    session_histories = []
    session_futures   = []
    session_labels    = []
    session_sizes     = []
    

    for key, subgroup in tqdm(data.groupby('session_id', sort = False)):
        # Split session in two
        half = int(np.floor(len(subgroup) / 2.0))
        first = subgroup[sess_features + track_feature_list].head(half).values
        second = subgroup[sess_features + track_feature_list].tail(subgroup.shape[0] - half)
        current_labels = second[label_index].astype(int).values

        second = second.values
        
        # Generate padding for session history
        padding = np.zeros((int(max_seq_len / 2.0) - first.shape[0], len(sess_features) + len(track_feature_list)))
        
        #stack all in a np array
        session_history = np.vstack([padding,first])
        
        # Generate padding for session future
        padding = np.zeros((int(max_seq_len / 2.0) - second.shape[0], len(track_feature_list)))
        session_future = np.vstack([second[:,-len(track_feature_list):],padding])
        
        # Save results
        session_histories.append(session_history)
        session_futures.append(session_future)
        
        s_l = list(current_labels)
        s_l = [x[0] for x in current_labels]
        times=int(max_seq_len / 2.0) - current_labels.shape[0]
        if times > 0:
           for i in range(times):
              np.array(s_l.append(0))
        session_labels.append(s_l)
        session_sizes.append(len(current_labels))
            
    return session_histories, session_futures, session_labels, session_sizes 
        


In [131]:
max_seq_len = 20

def process_sessions_train(data, sess_features, track_feature_list, label_index):
    session_histories = []
    session_futures   = []
    session_labels    = []
    session_sizes     = []
    
    for key, subgroup in tqdm(data.groupby('session_id', sort = False)):
        # Split session in two
        half = int(np.floor(len(subgroup) / 2.0))
        first = subgroup[sess_features + track_feature_list].head(half).values
        second = subgroup[sess_features + track_feature_list].tail(subgroup.shape[0] - half)
        
         
        # Generate labels
        current_labels = second[label_index].astype(int).values
        
        second = second.values
        # Generate padding for session history
        padding = np.zeros((int(max_seq_len / 2.0) - first.shape[0], len(sess_features) + len(track_feature_list)))
        
        #stack all in a np array
        session_history = np.vstack([padding,first])
        
        # Generate padding for session future
        padding = np.zeros((int(max_seq_len / 2.0) - second.shape[0], len(track_feature_list)))
        session_future = np.vstack([second[:,-len(track_feature_list):],padding])
        
        # Save results
        session_histories.append(session_history)
        session_futures.append(session_future)
        
        
        s_l = list(current_labels)
        s_l = [x[0] for x in current_labels]
        times=int(max_seq_len / 2.0) - current_labels.shape[0]
        if times > 0:
            for i in range(times):
                np.array(s_l.append(0))
        session_labels.append(s_l)
        session_sizes.append(len(current_labels))

    return session_histories, session_futures, session_labels, session_sizes 



In [132]:
max_seq_len = 20

def process_sessions_test(data, sess_features, track_feature_list, label_index):
    session_histories = []
    session_futures   = []
    session_labels    = []
    session_sizes     = []
    
    for key, subgroup in tqdm(data.groupby('session_id', sort = False)):
        # Split session in two
        half = int(np.floor(len(subgroup) / 2.0))
        first = subgroup[sess_features + track_feature_list].head(half).values
        second = subgroup[sess_features + track_feature_list].tail(subgroup.shape[0] - half)
        
        label_shape = second.shape[0]
        second = second.values
        # Generate padding for session history
        padding = np.zeros((int(max_seq_len / 2.0) - first.shape[0], len(sess_features) + len(track_feature_list)))
        
        #stack all in a np array
        session_history = np.vstack([padding,first])
        
        # Generate padding for session future
        padding = np.zeros((int(max_seq_len / 2.0) - second.shape[0], len(track_feature_list)))
        session_future = np.vstack([second[:,-len(track_feature_list):],padding])
        
        # Save results
        session_histories.append(session_history)
        session_futures.append(session_future)
        
        session_sizes.append(label_shape)

    return session_histories, session_futures, session_sizes


In [ ]:
sf = list(data.columns.values)
for el in ["track_id_clean", "session_id", "train_flag"]:
    sf.remove(el)

In [137]:
dataset.loc[150006:, 'train_flag'] = False

# Saving Dataset Feature-engineered

In [141]:
#Prepare training dataset and save it

train_dataset = dataset[dataset.train_flag == True]
history_train, future_train, labels_train, session_len_train = process_sessions_train(train_dataset, sf, tf_f, ["label"])
#Save

# Encode sessions as numpy arrays
history_train = np.asarray(history_train)
future_train = np.asarray(future_train)
labels_train = np.asarray(labels_train)
session_len_train = np.asarray(session_len_train)

# Dump to file
np.savez_compressed(processedpath + "/trainfeatures.npz",
    history_train  = history_train,
    future_train = future_train,
    labels_train = labels_train,
    session_len_train = session_len_train)













  0%|          | 0/8934 [00:00<?, ?it/s]











  0%|          | 1/8934 [00:00<21:57,  6.78it/s]











  0%|          | 26/8934 [00:00<15:30,  9.57it/s]











  1%|          | 51/8934 [00:00<11:00, 13.45it/s]











  1%|          | 74/8934 [00:00<07:52, 18.74it/s]











  1%|          | 95/8934 [00:00<05:43, 25.75it/s]











  1%|▏         | 116/8934 [00:00<04:12, 34.93it/s]











  2%|▏         | 135/8934 [00:00<03:13, 45.36it/s]











  2%|▏         | 153/8934 [00:00<02:31, 58.02it/s]











  2%|▏         | 176/8934 [00:00<01:57, 74.57it/s]











  2%|▏         | 199/8934 [00:01<01:33, 93.20it/s]











  3%|▎         | 228/8934 [00:01<01:14, 116.40it/s]











  3%|▎         | 252/8934 [00:01<01:03, 136.79it/s]











  3%|▎         | 275/8934 [00:01<01:04, 135.29it/s]











  3%|▎         | 297/8934 [00:01<00:56, 152.74it/s]











  4%|▎         | 324/8934 [00:01<00:49, 175.02it/s]











  4%|▍ 

 22%|██▏       | 1939/8934 [00:08<00:32, 212.63it/s]











 22%|██▏       | 1965/8934 [00:09<00:31, 223.40it/s]











 22%|██▏       | 1993/8934 [00:09<00:29, 237.35it/s]











 23%|██▎       | 2019/8934 [00:09<00:28, 243.46it/s]











 23%|██▎       | 2044/8934 [00:09<00:28, 242.64it/s]











 23%|██▎       | 2069/8934 [00:09<00:28, 236.92it/s]











 23%|██▎       | 2093/8934 [00:09<00:29, 230.37it/s]











 24%|██▎       | 2117/8934 [00:09<00:29, 231.90it/s]











 24%|██▍       | 2146/8934 [00:09<00:27, 245.28it/s]











 24%|██▍       | 2176/8934 [00:09<00:26, 259.02it/s]











 25%|██▍       | 2206/8934 [00:09<00:25, 267.21it/s]











 25%|██▌       | 2234/8934 [00:10<00:25, 259.31it/s]











 25%|██▌       | 2261/8934 [00:10<00:30, 215.51it/s]











 26%|██▌       | 2288/8934 [00:10<00:29, 228.16it/s]











 26%|██▌       | 2318/8934 [00:10<00:26, 245.28it/s]











 26%|██▋       | 2346/8934 [00:10<00:26,

 46%|████▌     | 4086/8934 [00:18<00:26, 185.30it/s]











 46%|████▌     | 4107/8934 [00:18<00:26, 182.94it/s]











 46%|████▌     | 4127/8934 [00:18<00:25, 185.80it/s]











 46%|████▋     | 4147/8934 [00:18<00:27, 175.52it/s]











 47%|████▋     | 4166/8934 [00:18<00:27, 173.65it/s]











 47%|████▋     | 4193/8934 [00:18<00:24, 193.69it/s]











 47%|████▋     | 4214/8934 [00:18<00:25, 185.66it/s]











 47%|████▋     | 4234/8934 [00:18<00:26, 174.92it/s]











 48%|████▊     | 4253/8934 [00:19<00:27, 170.63it/s]











 48%|████▊     | 4271/8934 [00:19<00:27, 171.98it/s]











 48%|████▊     | 4299/8934 [00:19<00:23, 193.22it/s]











 48%|████▊     | 4320/8934 [00:19<00:23, 195.53it/s]











 49%|████▊     | 4341/8934 [00:19<00:24, 187.43it/s]











 49%|████▉     | 4366/8934 [00:19<00:22, 202.42it/s]











 49%|████▉     | 4388/8934 [00:19<00:25, 180.96it/s]











 49%|████▉     | 4408/8934 [00:19<00:25,

 68%|██████▊   | 6035/8934 [00:27<00:11, 245.98it/s]











 68%|██████▊   | 6060/8934 [00:27<00:11, 246.94it/s]











 68%|██████▊   | 6085/8934 [00:27<00:11, 244.23it/s]











 68%|██████▊   | 6111/8934 [00:27<00:11, 247.56it/s]











 69%|██████▊   | 6136/8934 [00:27<00:11, 241.88it/s]











 69%|██████▉   | 6161/8934 [00:27<00:11, 239.95it/s]











 69%|██████▉   | 6187/8934 [00:27<00:11, 244.06it/s]











 70%|██████▉   | 6213/8934 [00:28<00:10, 248.20it/s]











 70%|██████▉   | 6238/8934 [00:28<00:10, 245.15it/s]











 70%|███████   | 6265/8934 [00:28<00:10, 249.62it/s]











 70%|███████   | 6294/8934 [00:28<00:10, 258.30it/s]











 71%|███████   | 6322/8934 [00:28<00:09, 262.35it/s]











 71%|███████   | 6349/8934 [00:28<00:10, 249.87it/s]











 71%|███████▏  | 6375/8934 [00:28<00:10, 252.25it/s]











 72%|███████▏  | 6403/8934 [00:28<00:09, 257.77it/s]











 72%|███████▏  | 6429/8934 [00:28<00:09,

 92%|█████████▏| 8188/8934 [00:36<00:02, 249.46it/s]











 92%|█████████▏| 8214/8934 [00:36<00:03, 234.81it/s]











 92%|█████████▏| 8238/8934 [00:36<00:02, 232.98it/s]











 93%|█████████▎| 8265/8934 [00:36<00:02, 242.82it/s]











 93%|█████████▎| 8292/8934 [00:36<00:02, 248.59it/s]











 93%|█████████▎| 8318/8934 [00:36<00:02, 250.03it/s]











 93%|█████████▎| 8344/8934 [00:36<00:02, 222.14it/s]











 94%|█████████▎| 8367/8934 [00:36<00:02, 214.65it/s]











 94%|█████████▍| 8390/8934 [00:37<00:02, 203.48it/s]











 94%|█████████▍| 8414/8934 [00:37<00:02, 212.31it/s]











 94%|█████████▍| 8441/8934 [00:37<00:02, 225.41it/s]











 95%|█████████▍| 8466/8934 [00:37<00:02, 232.01it/s]











 95%|█████████▌| 8490/8934 [00:37<00:01, 232.40it/s]











 95%|█████████▌| 8517/8934 [00:37<00:01, 241.77it/s]











 96%|█████████▌| 8543/8934 [00:37<00:01, 245.94it/s]











 96%|█████████▌| 8570/8934 [00:37<00:01,

In [142]:
#Prepare test dataset and save it
test_dataset = dataset.loc[dataset['train_flag'] == False]
history_test, future_test, session_len_test = process_sessions_test(test_dataset, sf, tf_f, ["label"])

#Save

# Encode sessions as numpy arrays
history_test = np.asarray(history_test)
future_test = np.asarray(future_test)
session_len_test = np.asarray(session_len_test)

# Dump to file
np.savez_compressed(processedpath + "/testfeatures.npz",
    history_test  = history_test,
    future_test = future_test,
    session_len_test = session_len_test)













  0%|          | 0/5112 [00:00<?, ?it/s]











  0%|          | 5/5112 [00:00<01:43, 49.11it/s]











  1%|          | 39/5112 [00:00<01:16, 66.06it/s]











  2%|▏         | 78/5112 [00:00<00:57, 87.97it/s]











  2%|▏         | 116/5112 [00:00<00:43, 114.13it/s]











  3%|▎         | 153/5112 [00:00<00:34, 143.77it/s]











  4%|▎         | 190/5112 [00:00<00:28, 175.36it/s]











  4%|▍         | 221/5112 [00:00<00:26, 185.08it/s]











  5%|▍         | 249/5112 [00:00<00:25, 190.85it/s]











  5%|▌         | 275/5112 [00:01<00:24, 199.40it/s]











  6%|▌         | 311/5112 [00:01<00:20, 229.54it/s]











  7%|▋         | 349/5112 [00:01<00:18, 259.94it/s]











  8%|▊         | 385/5112 [00:01<00:16, 283.19it/s]











  8%|▊         | 418/5112 [00:01<00:18, 257.72it/s]











  9%|▊         | 447/5112 [00:01<00:19, 243.52it/s]











  9%|▉         | 474/5112 [00:01<00:19, 240.13it/s]









